<a href="https://colab.research.google.com/github/zaelcovsky/workshop_deep_python_autumn_2023_not_my_group/blob/main/deep_python_vk_workshop_asyncpg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Асинхронное выполнение запросов к базе данных Postgres

В данном задании предлагается рассмотреть код, взаимодействующий с БД, и ускорить обработку большого числа запросов с помощью асинхронного программирования.

Для работы с БД предлагается использовать [докер](https://www.docker.com/) и подготовленный для воркшопа образ с БД. Поднять контейнер с БД можно так:

> ```bash
> docker run --rm -d -p 5432:5432 --name postgres \
>     -e POSTGRES_PASSWORD=admin \
>     -e POSTGRES_USER=admin \
>     -e POSTGRES_DB=products \
>     -e PGDATA=/var/lib/postgresql/pgdata \
>     dmitrylala/deep-python-vk:1
> ```

**Note:** Вот так можно выполнить healthcheck и проверить что postgres поднялся
> ```bash
> dmitry@dmitry:~$ docker exec -it postgres pg_isready -U admin
> /var/run/postgresql:5432 - accepting connections
>  ```

In [ ]:
# !pip install asyncpg

In [ ]:
from typing import List, Tuple
import time
import asyncio

import asyncpg

ModuleNotFoundError: ignored

In [ ]:
POSTGRES_PARAMS = {
    "host": "127.0.0.1",
    "port": 5432,
    "user": "admin",
    "database": "products",
    "password": "admin",
}

### Посмотрим на данные

In [ ]:
conn = await asyncpg.connect(**POSTGRES_PARAMS)

brand_query = "SELECT brand_id, brand_name FROM brand LIMIT 10"
results = await conn.fetch(brand_query)

for brand in results:
    print(f'id: {brand["brand_id"]}, name: {brand["brand_name"]}')

await conn.close()

id: 1, name: Levis
id: 2, name: Seven
id: 3, name: bar
id: 4, name: sure
id: 5, name: old
id: 6, name: discover
id: 7, name: everything
id: 8, name: dog
id: 9, name: surface
id: 10, name: machine


In [ ]:
conn = await asyncpg.connect(**POSTGRES_PARAMS)

product_query = "SELECT product_id, product_name FROM product LIMIT 10"
results = await conn.fetch(product_query)

for product in results:
    print(f'id: {product["product_id"]}, name: {product["product_name"]}')

print("\n")

sku_query = "SELECT sku_id, product_id FROM sku LIMIT 10"
results = await conn.fetch(sku_query)

for sku in results:
    print(f'id: {sku["sku_id"]}, name: {sku["product_id"]}')

await conn.close()

id: 1, name: catch reason authority shake discover analysis customer popular worker live
id: 2, name: save arm within big per member early my community own
id: 3, name: cause least image coach company enter anyone my size husband
id: 4, name: money baby war process similar Mr pressure positive science expect
id: 5, name: country pattern foot car set discussion all threat relate cost
id: 6, name: dark sister rise another policy whatever woman issue space family
id: 7, name: discussion value shoulder senior deep expert live beat could animal
id: 8, name: white participant physical factor summer a camera reflect cell high
id: 9, name: occur body our pick they laugh professor agreement message dog
id: 10, name: top collection with protect woman look page another strategy election


id: 1, name: 480
id: 2, name: 878
id: 3, name: 156
id: 4, name: 173
id: 5, name: 656
id: 6, name: 183
id: 7, name: 834
id: 8, name: 909
id: 9, name: 164
id: 10, name: 50


### Симуляция "тяжелого" популярного запроса

In [ ]:
# хотим получить все имеющиеся SKU для конкретного товара
product_query = \
"""
SELECT
p.product_id,
p.product_name,
p.brand_id,
s.sku_id,
pc.product_color_name,
ps.product_size_name
FROM product as p
JOIN sku as s on s.product_id = p.product_id
JOIN product_color as pc on pc.product_color_id = s.product_color_id
JOIN product_size as ps on ps.product_size_id = s.product_size_id
WHERE p.product_id = {product_id}"""


POSTGRES_POOL_PARAMS = {
    "min_size": 6,
    "max_size": 6,
    **POSTGRES_PARAMS,
}

In [ ]:
async def query_product(pool, product_id):
    async with pool.acquire() as connection:
        return await connection.fetchrow(product_query.format(product_id=product_id))


async with asyncpg.create_pool(**POSTGRES_POOL_PARAMS) as pool:
    res = await query_product(pool, product_id=20)
    print(res)

<Record product_id=20 product_name='end factor program former grow Democrat decision lawyer better already' brand_id=40 sku_id=14 product_color_name='Blue' product_size_name='Medium'>


In [ ]:
product_ids = list(range(1000)) * 100

In [ ]:
start = time.perf_counter()

async with asyncpg.create_pool(**POSTGRES_POOL_PARAMS) as pool:
    for product_id in product_ids:
        _ = await query_product(pool, product_id=product_id)

print(time.perf_counter() - start)

53.657701909000025


### Задание: максимально ускорить выполнение 100к запросов, используя асинхронность